- 가온 차트에는 곡마다 멜론으로 연결되는 링크가 존재
- 그 링크를 이용해 앨범 정보 (앨범 좋아요 / 앨범 평점 / 평점 남긴 사람 수 / 앨범 댓글 수 / song_id) 와 곡 정보 (장르 / 가사) 를 크롤링하고자 함

## 가온 차트 크롤링
순위 / 제목 / 가수 / 가온지수 / album_id (javascript url 형식)

In [16]:
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import pandas as pd

Year = ['2018', '2019']
Month = ['03', '06', '09', '12']
rankings = []
titles = []
singers = []
counts = []
album_id = []

for Y in tqdm(Year):
    for M in Month:
        url = "http://gaonchart.co.kr/main/section/chart/online.gaon?nationGbn=T&serviceGbn=ALL&targetTime=%s&hitYear=%s&termGbn=month" %(M,Y) 
        req = requests.get(url)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')
        gaon_chart = soup.find_all('tr')

        for g in gaon_chart:
            if g.find('td', {"class": 'ranking'}) is not None:
                # 순위 크롤링
                ranking = g.find('td', {"class": 'ranking'}).text
                rankings.append(ranking)
                # 제목 크롤링
                title = g.find("td", {"class": "subject"}).p.text.strip()
                titles.append(title)
                # 가수 크롤링
                singer = g.find("p", {"class": "singer"}).text.split('|')[0]
                singers.append(singer)
                # 가온지수 크롤링
                count = int(g.find("td", {"class": "count"}).p.text.replace(',',''))
                counts.append(count)
                # 앨범 아이디 크롤링
                play = g.find_all("td", {"class": "play"})
                for p in play:
                    link_tag = p.find_all('a')[0]
                    if 'href' in link_tag.attrs:
                        album_id.append(link_tag.attrs['href'].split('"')[3])

In [17]:
# 데이터 합치기

chart = pd.DataFrame( data = {'ranking': rankings, 'title': titles, 'singer': singers, 'count': counts, 'album_id' : album_id}) ; chart                    

,ranking,title,singer,count,album_id
0,1,별이 빛나는 밤,마마무 (Mamamoo),219015952,1477451
1,2,사랑을 했다 (LOVE SCENARIO),iKON,212503498,1436717
2,3,꽃 길,빅뱅,196811315,1482894
3,4,뿜뿜,모모랜드 (MOMOLAND),176579938,1422062
4,5,Jenga (Feat. Gaeko),헤이즈 (Heize),176316746,1477396
...,...,...,...,...,...
1195,196,비가 내리는 날에는,윤하,11921003,1796299
1196,197,고백,양다일,11847774,1677973
1197,198,YES or YES,TWICE,11722790,1687302
1198,199,180도,벤 (Ben),11610749,1695247


## 가온 album_id를 이용한 멜론 앨범 정보 크롤링 함수
앨범 좋아요 / 앨범 평점 / 평점 남긴 사람 수 / 앨범 댓글 수 / song_id 

In [20]:
from selenium import webdriver
from re import sub

punc_str = '[,.?!;:`]'

def detail_info(album_id, title):
    path = "C:/Users/user/chromedriver_win32/chromedriver"
    driver = webdriver.Chrome(path)
    url = "http://www.gaonchart.co.kr/main/section/chart/ReturnUrl.gaon?serviceGbn=ALL&seq_company=3715&seq_mom=%s" %(str(album_id))
    driver.get(url)
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    
    album_like = soup.select_one("#d_like_count").text.replace('총건수', '').strip().replace(',', '')
    album_grade = soup.select_one("#gradPointLayer").text
    album_grade_count = soup.select_one("#gradCountLayer").text.replace('명', '').replace(',','')
    album_comment = soup.select_one("a span#revCnt").text.replace('개', '')
    
    song_list = list(soup.select("a.btn.button_icons.type03.song_info"))
    for s in song_list:
        t = s.attrs['title']
        t = sub(punc_str,'', t.replace("'", ''))
        if(t.startswith(title[0:2]) or t.lower() in sub(punc_str,'', title.lower()) or sub(punc_str,'', title.lower()) in t.lower()):
            song_id = s.attrs['href'].split("'")[1] # song_id 저장
            driver.close() 
            return album_like, album_grade, album_grade_count, album_comment, song_id
        

detail_info(1477451, "별이 빛나는 밤")

('25272', '4.1', '18483', '34,893', '30937275')

## song_id를 이용한 곡 정보 크롤링 함수
장르 / 가사

In [18]:
import re
from fake_useragent import UserAgent
ua = UserAgent()

def get_lyrics(song_id):
    url = "http://www.melon.com/song/detail.htm?songId=%s"  %(str(song_id))
    headers = {'User-Agent' : ua.random}
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    # song_like = soup.select_one("span#d_like_count").text.replace('총건수', '').strip().replace(',', '')
    element = soup.select("dl.list dd")
    genre = element[2].text # 장르 크롤링
    txt = soup.select_one('#d_video_summary') # 가사 크롤링
    line = list(str(txt).split("<br>"))
    rep = re.compile("(<div.+-->|</div.?>|</?br/?>)") # 속성값 제거
    data = rep.sub(" ",str(txt))
    lyrics = data.strip()
    
    return genre, lyrics

get_lyrics(30937275)

('댄스',
 '손끝이 시리더니 벌써 봄이 왔네 꿈같은 바람이 불어 곳곳에 여느 때 와는 다른 듯한 이 기분이 반가워 내일은 좀 다른 날이 되려나 어둠의 그림자 깊은 이 밤에 남겨진 달이 달이 밝아와 빛을 그리고 있어 내 몸에서 널 뺀 만큼 욕조에 물이 미련이 되어 넘쳐흘러 한숨이 가득 그 공간을 채워 새벽이 찾아오면 혼자 남겨질 때면 Starry night 별이 빛나는 밤 네가 휘휘휘 불어와 너와 나 나눈 수많은 밤 나를 취하게 만들어 별이 빛나는 밤 야야야야이야 별이 빛나는 밤 야야야야야야야 별이 빛나는 밤 겨울이 지나고 또다시 꽃은 피고 따뜻한 공기 냄새가 날 감싸 달라진 거 없는 똑같은 일상이 반복돼 그래도 내일을 기다려봐 하루에 끝에서 이 자리에 남아서 날이 날이 새도록 너를 그리고 있어 Do you want me to leave No 난 그 자리 그대로 까만 밤하늘에 가려 잠시 보이지 않는 거야 새벽이 찾아오면 혼자 남겨질 때면 Starry night 별이 빛나는 밤 네가 휘휘휘 불어와 너와 나 나눈 수많은 밤 나를 취하게 만들어 별이 빛나는 밤 야야야야이야 별이 빛나는 밤 야야야야야야야 별이 빛나는 밤 Starry Starry Night 네가 없는 밤 가슴이 공허해 우리의 풍경이 저물어가 현실을 부정해 눈을 감아도 네가 보여 왜 아른거려 Leave me alone 무너져 가네 Starry night 별이 빛나는 밤 네가 휘휘휘 불어와 너와 나 나눈 수많은 밤 나를 취하게 만들어 별이 빛나는 밤 야야야야이야 별이 빛나는 밤 야야야야야야야 별이 빛나는 밤')

## 멜론 크롤링 함수 적용


In [ ]:
chart['result'] = chart.apply(lambda x:detail_info(x['album_id'], x['title']), axis = 1)

In [ ]:
chart[['album_like', 'album_grade', 'album_grade_count', 'album_comment', 'song_id']] = pd.DataFrame(chart['result'].tolist())

In [ ]:
chart.drop(['result'], axis =1)

In [ ]:
chart['result'] = chart.apply(lambda x:get_lyrics(x['song_id']), axis = 1)

In [ ]:
chart[['genre', 'lyrics']] = pd.DataFrame(chart['result'].tolist())

In [ ]:
chart.drop(['result'], axis =1)

In [ ]:
# 엑셀 파일로 저장

chart.to_excel("1차 데이터.xlsx") 